# Eshelby Example

## Rotation functions

First we will define rotation functions for $Q_{ij}$ and $R_\sigma$

In [1]:
import numpy as np

def qij_x(theta):
    """
    rotation tensor about x-axis by some theta
    input: theta (angle in radians)
    output: qij (3x3 rotation tensor)
    """
    qij = np.array([[1,0,0],
                   [0,np.cos(theta),-np.sin(theta)],
                   [0,np.sin(theta),np.cos(theta)]])
    return qij

def qij_y(theta):
    """
    rotation tensor abo|ut y-axis by some theta
    input: theta (angle in radians)
    output: qij (3x3 rotation tensor)
    """
    qij = np.array([[np.cos(theta), 0, np.sin(theta)],
                   [0,1,0],
                   [-np.sin(theta),0,np.cos(theta)]])
    return qij

def qij_z(theta):
    """
    rotation tensor about z-axis by some theta
    input: theta (angle in radians)
    output: qij (3x3 rotation tensor)
    """
    qij = np.array([[np.cos(theta),-np.sin(theta),0],
                   [np.sin(theta),np.cos(theta),0],
                   [0,0,1]])
    return qij

def R_sigma(q):
    """
    rotation matrix for engineering notation given some rotation tensor, qij
    note: uses convention for sigma = [s11, s22, s33, s23, s13, s12]
    input: q (3x3 rotation tensor)
    output: R_s (6x6 rotation matrix)
    """
    R_s = np.array([[q[0,0]**2,q[0,1]**2,q[0,2]**2,q[0,1]*q[0,2],q[0,0]*q[0,2], q[0,0]*q[0,1]],
                       [q[1,0]**2,q[1,1]**2,q[1,2]**2,q[1,1]*q[1,2],q[1,0]*q[1,2], q[1,0]*q[1,1]],
                       [q[2,0]**2,q[2,1]**2,q[2,2]**2,q[2,1]*q[2,2],q[2,0]*q[2,2], q[2,0]*q[2,1]],
                       [2*q[1,0]*q[2,0], 2*q[1,1]*q[2,1], 2*q[1,2]*q[2,2],q[1,2]*q[2,1]+q[1,1]*q[2,2], q[1,2]*q[2,0]+q[1,0]*q[2,2], q[1,1]*q[2,0]+q[1,0]*q[2,1]],
                       [2*q[0,0]*q[2,0], 2*q[0,1]*q[2,1], 2*q[0,2]*q[2,2],q[0,2]*q[2,1]+q[0,1]*q[2,2], q[0,2]*q[2,0]+q[0,0]*q[2,2], q[0,1]*q[2,0]+q[0,0]*q[2,1]],
                       [2*q[0,0]*q[1,0], 2*q[0,1]*q[1,1], 2*q[0,2]*q[1,2],q[0,2]*q[1,1]+q[0,1]*q[1,2], q[0,2]*q[1,0]+q[0,0]*q[1,2], q[0,1]*q[1,0]+q[0,0]*q[1,1]]])
    return R_s

## Eshelby Tensor

Now we calculate parameters for the Eshelby Tensor

In [12]:
nu_m = 0.40 #matrix poisson's ratio
s = 100. #fiber aspect ratio
I1 = (2.*s/(s**2.-1.)**1.5)*(s*(s**2.-1.)**.5-np.arccosh(s))
Q = 3./(8.*(1-nu_m))
R = (1.-2.*nu_m)/(8.*(1.-nu_m))
T = Q*(4.-3.*I1)/(3*(s**2.-1.))
I3 = 4.-2.*I1

And now we find the Eshelby tensor.

In [13]:
S = np.zeros((6,6))
S[0,0] = Q + R*I1 + 0.75*T
S[1,1] = S[0,0]
S[2,2] = 4./3.*Q + R*I3 + 2.*s**2.*T
S[0,1] = Q/3. - R*I1 + 4.*T/3.
S[1,0] = S[0,1]
S[0,2] = -R*I1 - s**2*T
S[1,2] = S[0,2]
S[2,0] = -R*I3 - T
S[2,1] = S[2,0]
S[5,5] = Q/3 + R*I1 + T/4
S[3,3] = 2*R - I1*R/2 - (1+s**2)*T/2
S[4,4] = S[3,3]

We now rotate the Eshelby tensor into the $\pm 45^\circ$ directions. If we want fiber directions to align in the x-axis (instead of the z-axis as is defined), we will need two rotations, one to put the fibers in the x-direction, and one to rotate by $\pm 45^\circ$.

This rotation could be done multiple ways, but I will do it by first rotation $90^\circ$ about the x2-axis, then rotating $\pm45^\circ$ about the new x1' axis. As we found from our example in Lecture 2 (slide 35), one effective Q for this transformation can be found by multiplying $Q^2 Q^1$, however in this case our fiber is already rotated, so we are solving this inverse problem. We can do this by multiplying $(Q^1)^T (Q^2)^T$

In [14]:
Q1 = qij_y(np.pi/2)
Q2_45 = qij_x(np.pi/4) #plus 45 rotation
Q2_m45 = qij_x(-np.pi/4) #minus 45 rotation

Q_45 = np.dot(Q1.T,Q2_45.T) #matrix multiplication in python
Q_m45 = np.dot(Q1.T,Q2_m45.T)

R_45 = R_sigma(Q_45)
R_m45 = R_sigma(Q_m45)

S_45 = np.dot(R_45,np.dot(S,R_45.T))
S_m45 = np.dot(R_m45,np.dot(S,R_m45.T))

# Stiffness Matrix

Now we need to build the stiffness matrices for $C_f$ and $C_m$ to find the strain concentration tensors.
For simplicity, we will assume isotropic material properties.
If the matrix were not isotropic, we would need to use a different formula to calculate the Eshelby tensor, but that is not the case here.

In [15]:
Ef = 200. #GPa
nu_f = 0.2
Em = 10. #GPa
#nu_m defined above

Cf = np.zeros((6,6))
Cf[0:3,0:3] = nu_f*np.ones((3,3))
Cf = Cf + (1.-2.*nu_f)*np.eye(6)
Cf[3:,3:] = Cf[3:,3:]/2.
Cf = Ef/((1.+nu_f)*(1.-2.*nu_f))*Cf

Cm = np.zeros((6,6))
Cm[0:3,0:3] = nu_m*np.ones((3,3))
Cm = Cm + (1.-2.*nu_m)*np.eye(6)
Cm[3:,3:] = Cm[3:,3:]/2.
Cm = Em/((1+nu_m)*(1.-2.*nu_m))*Cm

# Strain Concentration Tensor
We can now find the strain concentration tensors
$$
A^{45} = \left[I + S^{45} (C_m)^{-1}(C_f-C_m) \right]^{-1}
$$
$$
A^{-45} = \left[I + S^{-45} (C_m)^{-1}(C_f-C_m) \right]^{-1}
$$

In [16]:
A_45 = np.linalg.inv(np.eye(6)+np.dot(np.dot(S_45,np.linalg.inv(Cm)),(Cf-Cm)))
A_m45 = np.linalg.inv(np.eye(6)+np.dot(np.dot(S_m45,np.linalg.inv(Cm)),(Cf-Cm)))

## Effective Stiffness
And, finally, we can use the strain concentration tensors to calculate the effective stiffness.
$$
C = C_m + v^{45}(C_f-C_m)A^{45} + v^{-45}(C_f-C_m)A^{-45}
$$

In [17]:
v_45 = 0.3 #volume fraction of +45
v_m45 = 0.3 #volume fraction of -45
C = Cm + v_45*np.dot((Cf-Cm),A_45) + v_m45*np.dot((Cf-Cm),A_m45)

To interpret the stiffness, we convert it to compliance and find youngs modulus in each direction

In [18]:
S = np.linalg.inv(C)
print 1/S[0,0]
print 1/S[1,1]
print 1/S[2,2]

48.8481826234
48.8481826234
29.6152006322


# Mori Tanaka

We can follow identical logic as above to use the Mori-Tanaka method, the only difference is in calculating the strain concentration tensor.

We will calculate the Mori-Tanaka strain concentration tensor in terms of the Eshelby strain concentration tensor.
Recall that for multiple inclusions we first calculate the total Eshelby strain concentration tensor
$$
A^{Tot} = \sum V_i A^E_i
$$
then we can calculate each individual strain concentration as
$$
A^{MT}_i = A^E_i\left[(1-V_f) I + A^{Tot} \right]^{-1}
$$

In [19]:
#mori-tanaka strain concentration tensors
v_m = 1.0 -v_45 - v_m45 #matrix volume fraction
A_tot = A_45*v_45 + A_m45*v_m45
A_mt_45 = np.dot(A_45,np.linalg.inv(A_tot+(1-v_45)*np.eye(6)))
A_mt_m45 = np.dot(A_m45,np.linalg.inv(A_tot+(1-v_m45)*np.eye(6)))

And stiffness is calculated in the same way, using the Mori-Tanaka strain concentration tensors

In [20]:
C_mt = Cm + v_45*np.dot((Cf-Cm),A_mt_45) + v_m45*np.dot((Cf-Cm),A_mt_m45)

And now we compare the Young's modulus in the axial directions

In [21]:
S_mt = np.linalg.inv(C_mt)
print 1/S_mt[0,0]
print 1/S_mt[1,1]
print 1/S_mt[2,2]

52.9100788186
52.9100788186
33.1951195327


# Halpin-Tsai

The Halpin-Tsai model does not directly build a stiffness matrix, and thus cannot be easily used to model composites with off-axis orientation, but we can easily compare stiffness for varying fiber aspect ratios.

Here, we will use the same composite properties as in the Eshelby and Mori-Tanaka models and calculate axial and transverse stiffness.

In [22]:
zeta = 2*s
eta = (Ef/Em-1)/(Ef/Em+zeta)
f = 0.20 # 20% volume fraction
Ec_20 = Em*(1+zeta*eta*f)/(1-eta*f)
Ec_20

45.32839962997225

We can compare this with Eshelby and Mori-Tanaka uni-directional models by re-calculating for uni-directional fiber orientation

In [23]:
A_ud = np.linalg.inv(np.eye(6)+np.dot(np.dot(S,np.linalg.inv(Cm)),(Cf-Cm)))
C_ud_esh = Cm + f*np.dot((Cf-Cm),A_ud)
S_ud_esh =np.linalg.inv(C_ud_esh)
1/S_ud_esh[0,0]

35.268246392675167

For Eshelby and for Mori-Tanaka we find:

In [24]:
v_m = 1.0 -f #matrix volume fraction
A_tot = A_ud*f
A_mt_ud = np.dot(A_ud,np.linalg.inv(A_tot+(1-f)*np.eye(6)))
C_mt_ud = Cm + f*np.dot((Cf-Cm),A_mt_ud)
S_mt_ud =np.linalg.inv(C_mt_ud)
1/S_mt_ud[0,0]

37.212885730597918